<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#还原" data-toc-modified-id="还原-1">还原</a></span></li><li><span><a href="#编译" data-toc-modified-id="编译-2">编译</a></span></li><li><span><a href="#验证" data-toc-modified-id="验证-3">验证</a></span></li></ul></div>

# <div align='center'> 将编译后的pb2文件转换为proto文件</div>

In [2]:
import os
import sys
from google.protobuf.descriptor import FieldDescriptor as FD
from k12libs.utils.nb_easy import k12ai_get_app_dir
from k12libs.utils.nb_easy import K12AI_NBDATA_ROOT

In [3]:
proto_outdir = os.path.join(K12AI_NBDATA_ROOT, 'vulkan/protos/layers')
if not os.path.exists(proto_outdir):
    os.makedirs(proto_outdir)
sys.path.append(k12ai_get_app_dir('cv'))
sys.path.append(K12AI_NBDATA_ROOT)

## 还原

In [4]:
from vulkan.protos.layers.conv_pb2 import ConvLayer
from vulkan.protos.layers.activations_pb2 import ActLayer
from vulkan.protos.layers.dropout_pb2 import DropoutLayer
from vulkan.protos.layers.linear_pb2 import LinearLayer
from vulkan.protos.layers.padding_pb2 import PaddingLayer
from vulkan.protos.layers.pooling_pb2 import PoolLayer
from vulkan.protos.layers.norm_pb2 import NormLayer
from vulkan.protos.layers.basefunc_pb2 import BaseFuncLayer
from vulkan.protos.layers.vulkan_pb2 import VulkanLayer
from vulkan.protos.layers.layers_pb2 import Layer
from vulkan.protos.skeletons.skeleton_pb2 import skeletons

classes = [
    ConvLayer,
    ActLayer,
    DropoutLayer,
    LinearLayer,
    PaddingLayer,
    PoolLayer,
    NormLayer,
    BaseFuncLayer,
    VulkanLayer
]

In [5]:
type2string = {
    FD.TYPE_BOOL: 'bool',
    FD.TYPE_BYTES: 'bytes',
    FD.TYPE_DOUBLE: 'double',
    FD.TYPE_ENUM: 'enum',
    FD.TYPE_FIXED32: 'fixed32',
    FD.TYPE_FIXED64: 'fixed64',
    FD.TYPE_FLOAT: 'float',
    FD.TYPE_GROUP: 'group',
    FD.TYPE_INT32: 'int32',
    FD.TYPE_INT64: 'int64',
    FD.TYPE_SFIXED32: 'sfixed32',
    FD.TYPE_SFIXED64: 'sfixed64',
    FD.TYPE_SINT32: 'sint32',
    FD.TYPE_SINT64: 'sint64',
    FD.TYPE_STRING: 'string',
    FD.TYPE_UINT32: 'uint32',
    FD.TYPE_UINT64: 'uint64'
}

def class2proto(cls, space=0):
    obj = cls()
    proto = '%smessage %s {\n' % (' ' * space, cls.__name__)
    space += 4
    for field in obj.DESCRIPTOR.fields:
        # print(field.name, field.id, field.index, field.label, field.number,
        #         field.type, field.message_type, field.cpp_type, field.enum_type,
        #         field.has_options, field.has_default_value, field.default_value)

        msg_type = field.message_type

        if field.type == FD.TYPE_MESSAGE:
            proto += class2proto(msg_type._concrete_class, space)
            str_type = msg_type._concrete_class.__name__
        elif field.type == FD.TYPE_ENUM:
            str_type = field.enum_type.name
            proto += '%senum %s {\n' % (' ' * space, str_type)
            for idx, val in enumerate(field.enum_type.values):
                proto += f'{" "*(space + 4)}{val.name} = {idx};\n'
            proto += '%s}\n' % (' ' * space)
            default_value = field.enum_type.values[field.default_value].name
        else:
            str_type = type2string[field.type]
            default_value = field.default_value
            
        default = ''
        if field.has_default_value:
            default = f' [default = {default_value}]'

        label = 'required'
        if field.label == FD.LABEL_REPEATED:
            label = 'repeated'
        elif field.label == FD.LABEL_OPTIONAL:
            label = 'optional'

        proto += f'{" "*space}{label} {str_type} {field.name} = {field.number}{default};\n'
    proto += '%s}\n' % (' ' * (space - 4))
    return proto

In [6]:
for cls in classes:
    print('-'*20, cls.DESCRIPTOR.file.name, '-'*20)
    protostr = 'syntax = "proto2";\n\n{}'.format(class2proto(cls))
    print(protostr)
    proto_file = os.path.basename(cls.DESCRIPTOR.file.name)
    with open(os.path.join(proto_outdir, proto_file), 'w') as fw:
        fw.write(protostr)

-------------------- vulkan/protos/layers/conv.proto --------------------
syntax = "proto2";

message ConvLayer {
    optional string name = 1;
    repeated string inputs = 2;
    repeated string outputs = 3;
    enum ConvMode {
        CONV1D = 0;
        CONV2D = 1;
        CONV3D = 2;
        TRANSCONV1D = 3;
        TRANSCONV2D = 4;
        TRANSCONV3D = 5;
    }
    optional ConvMode layer_mode = 4;
    message conv_parameter {
        optional string in_channels = 1;
        optional string out_channels = 2;
        repeated string kernel_size = 3;
        repeated string stride = 4;
        repeated string padding = 5;
        repeated string dilation = 6;
        optional string groups = 7;
        optional string bias = 8;
    }
    optional conv_parameter layer_params = 5;
    optional string layer_builder = 6;
}

-------------------- vulkan/protos/layers/activations.proto --------------------
syntax = "proto2";

message ActLayer {
    optional string name = 1;
    repeated s

In [ ]:
# print(class2proto(Layer))

In [ ]:
# print(class2proto(skeletons))

----------------------------

## 编译

In [ ]:
from k12libs.utils.nb_easy import k12ai_restart_kernel
k12ai_restart_kernel()

In [7]:
import sys
from k12libs.utils.nb_easy import K12AI_NBDATA_ROOT

sys.path.append(K12AI_NBDATA_ROOT)

In [8]:
!apt install protobuf-compiler -y
!cd {K12AI_NBDATA_ROOT}; protoc --python_out=.  vulkan/protos/layers/*.proto

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libprotobuf10 libprotoc10
The following NEW packages will be installed:
  libprotobuf10 libprotoc10 protobuf-compiler
0 upgraded, 3 newly installed, 0 to remove and 88 not upgraded.
Need to get 1242 kB of archives.
After this operation, 4942 kB of additional disk space will be used.
Get:1 http://mirrors.intra.didiyun.com/ubuntu bionic/main amd64 libprotobuf10 amd64 3.0.0-9.1ubuntu1 [651 kB]
Get:2 http://mirrors.intra.didiyun.com/ubuntu bionic/main amd64 libprotoc10 amd64 3.0.0-9.1ubuntu1 [566 kB]
Get:3 http://mirrors.intra.didiyun.com/ubuntu bionic/universe amd64 protobuf-compiler amd64 3.0.0-9.1ubuntu1 [24.5 kB]
Fetched 1242 kB in 0s (22.2 MB/s)             
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

-----------------------

## 验证

In [10]:
from vulkan.protos.layers.conv_pb2 import ConvLayer
from vulkan.protos.layers.activations_pb2 import ActLayer
from vulkan.protos.layers.dropout_pb2 import DropoutLayer
from vulkan.protos.layers.linear_pb2 import LinearLayer
from vulkan.protos.layers.padding_pb2 import PaddingLayer
from vulkan.protos.layers.pooling_pb2 import PoolLayer
from vulkan.protos.layers.norm_pb2 import NormLayer
from vulkan.protos.layers.basefunc_pb2 import BaseFuncLayer
from vulkan.protos.layers.vulkan_pb2 import VulkanLayer
from google.protobuf import text_format

layers_def = {
"conv": (ConvLayer, r"""
    name : "conv1"
    layer_mode : CONV2D 
    inputs : "layer1"
    outputs : "layer3"
    layer_params {
        in_channels : "3"
        out_channels : "6"
        kernel_size : "2"
        kernel_size : "3"
        bias : "True"
    }
    layer_builder : "NNTorchLayer"
    """),
"pool": (PoolLayer, r"""
    name : "pool1"
    layer_mode : MAXPOOL2D
    inputs : "layer2"
    outputs : "layer3"
    layer_params {
        kernel_size : "2"
        kernel_size : "2"
        padding : "1"
        return_indices : "False"
    }
    layer_builder : "NNTorchLayer"
    """),
"linear": (LinearLayer, r"""
    name : "linear1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : LINEAR
    layer_params {
        in_features : "20"
        out_features : "30"
        bias : "False"
    }
    layer_builder : "NNTorchLayer"
    """),
"dropout": (DropoutLayer, r"""
    name : "dropout1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : DROPOUT
    layer_params {
        inplace : "True"
        p : "0.6"
    }
    layer_builder : "NNTorchLayer"
    """),
"relu": (ActLayer, r"""
    name : "relu1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : RELU
    layer_params {
        inplace : "False"
    }
    layer_builder : "NNTorchLayer"
    """),
"reshape": (VulkanLayer, r"""
    name : "reshape1"
    inputs : "layer1"
    layer_mode : RESHAPE
    layer_params {
        target_shape : "4096"
    }
    layer_builder : "NNTorchLayer"
    """),
"cat": (BaseFuncLayer, r"""
    name : "cat1"
    inputs : "layer1"
    inputs : "layer2"
    layer_mode : CAT
    layer_params {
        dim : "0"
    }
    layer_builder : "TorchFuncLayer"
    """),
"norm": (NormLayer, r"""
    name : "bn1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : BATCHNORM2D
    layer_params {
        num_features : "36"
    }
    layer_builder : "NNTorchLayer"
    """),
"flatten": (VulkanLayer, r"""
    name : "flatten1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : FLATTEN
    layer_params : {
        start_dim : "1"
        end_dim : "-1"
    }
    layer_builder : "NNTorchLayer"
    """),
"add": (BaseFuncLayer, r"""
    name : "add1"
    inputs : "layer1"
    inputs : "layer2"
    layer_mode : ADD
    layer_builder : "TorchFuncLayer"
    """),
"padding": (PaddingLayer, r"""
    name : "pad1"
    inputs : "layer1"
    layer_mode : ZEROPAD2D
    layer_params : {
        padding : "1"
        padding : "1"
    }
    layer_builder : "NNTorchLayer"
    """)
}

In [11]:
for layer, (cls, netdef) in layers_def.items():
    print('-'*20, cls.DESCRIPTOR.file.name, '-'*20)
    print(netdef)
    proto = text_format.Merge(netdef, cls())
    print('-'*10, layer, '-'*10, '\n')
    print(proto)

-------------------- vulkan/protos/layers/conv.proto --------------------

    name : "conv1"
    layer_mode : CONV2D 
    inputs : "layer1"
    outputs : "layer3"
    layer_params {
        in_channels : "3"
        out_channels : "6"
        kernel_size : "2"
        kernel_size : "3"
        bias : "True"
    }
    layer_builder : "NNTorchLayer"
    
---------- conv ---------- 

name: "conv1"
inputs: "layer1"
outputs: "layer3"
layer_mode: CONV2D
layer_params {
  in_channels: "3"
  out_channels: "6"
  kernel_size: "2"
  kernel_size: "3"
  bias: "True"
}
layer_builder: "NNTorchLayer"

-------------------- vulkan/protos/layers/pooling.proto --------------------

    name : "pool1"
    layer_mode : MAXPOOL2D
    inputs : "layer2"
    outputs : "layer3"
    layer_params {
        kernel_size : "2"
        kernel_size : "2"
        padding : "1"
        return_indices : "False"
    }
    layer_builder : "NNTorchLayer"
    
---------- pool ---------- 

name: "pool1"
inputs: "layer2"
output